In [3]:
# **Importing libraries**
import pandas as pd
import numpy as np

In [17]:
#Uploading the query
df_join=pd.read_csv("Master_project_ab_testing.csv") 
df_join

,id,country,gender,group,join_dt,device,dt,device.1,spent
0,1010340,USA,F,B,2023-01-30,A,NaN,NaN,0.0
1,1020252,BRA,M,A,2023-01-27,A,NaN,NaN,0.0
2,1017309,BRA,M,A,2023-01-28,A,NaN,NaN,0.0
3,1002379,BRA,F,A,2023-02-03,A,NaN,NaN,0.0
4,1004713,TUR,NaN,B,2023-01-31,A,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...
49077,1048582,GBR,M,B,2023-01-28,I,NaN,NaN,0.0
49078,1028525,FRA,F,B,2023-01-28,A,NaN,NaN,0.0
49079,1029872,USA,M,B,2023-01-30,A,2023-01-30,A,23.0
49080,1034053,DEU,F,A,2023-01-29,A,NaN,NaN,0.0


In [8]:
#Installing the library
!pip install seaborn

In [21]:
df_join.group.value_counts()

group
B    24680
A    24402
Name: count, dtype: int64

In [15]:
import seaborn as sns

In [19]:
#Creating a fucntion that will build  graphs showing the detribution of A,B groups by feature
def plot_segments_distribution(df, segment_columns, test_factor):
    for segment in segment_columns:
        aggregated_data = df.groupby(by = [test_factor, segment])['id'].count().reset_index()
        sns.catplot(x = segment, 
                    y = 'id', 
                    hue = test_factor, 
                    data = aggregated_data, 
                    kind = "bar", 
                    height = 4, 
                    aspect = 1.5) 
plot_segments_distribution(df_join, #dataframe
                           ["device_g", "country","gender","conversion_flag"], #segments
                           "test_group") #test-factor

KeyError: 'test_group'

In [18]:
plot_segments_distribution(df_join, #dataframe
                           ["device_g", "country","gender","conversion_flag"], #segments
                           "test_group") #test-factor

KeyError: 'test_group'

In [ ]:
#Nice! The destribution looks good

In [ ]:
## Using the Chi-square test for conversion
!pip install statsmodels

In [ ]:
metrics_CR = df_join.groupby('test_group', as_index=False).agg({'id': 'count', 'conversion_flag': 'sum'})
metrics_CR['conversion_rate'] = round(metrics_CR['conversion_flag'] / metrics_CR['id'] * 100, 2)
metrics_CR

In [ ]:
import statsmodels.stats.proportion as proportion


In [ ]:
chi2stat, pval, table = proportion.proportions_chisquare(metrics_CR['conversion_flag'], metrics_CR['id']) 

In [ ]:
alpha = 0.05

In [ ]:
print(round(pval,5))

In [ ]:
print(pval < alpha)

In [ ]:
metrics_AOV = df_join.groupby('test_group', as_index=False).agg({'conversions_num': 'sum', 'sum_spent': 'sum'})
metrics_AOV['AOV'] = round(metrics['sum_spent'] / metrics['conversions_num'], 2)
metrics_AOV

In [ ]:
!pip install bootstrapped

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import bootstrapped.compare_functions as bs_compare

In [ ]:
??bs.bootstrap_ab

In [ ]:
test = df_join[df_join['test_group'] == 'B']['sum_spent'].dropna().values

In [ ]:
ctrl = df_join[df_join['test_group'] == 'A']['sum_spent'].dropna().values

In [ ]:
boot_results = bs.bootstrap_ab(test = test, ctrl = ctrl
                              , stat_func = bs_stats.mean
                              , compare_func = bs_compare.difference
                              , return_distribution=True)

In [ ]:
boot_results

In [ ]:
borders = np.percentile(boot_results, [2.5, 97.5]) # считаем перцентили
borders

In [ ]:
metrics_all = df_join.groupby('test_group', as_index=False).agg({'id': 'count', 'conversion_flag': 'sum','conversions_num': 'sum', 'sum_spent': 'sum'})
metrics_all['conversion_rate'] = round(metrics_CR['conversion_flag'] / metrics_CR['id'] * 100, 2)
metrics_all['AOV'] = round(metrics_all['sum_spent'] / metrics_all['conversions_num'], 2)
metrics_all['average_user_spent'] = round(metrics_all['sum_spent'] / metrics_all['id'], 2)


metrics_all